In [1]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, accuracy_score
import csv

In [2]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd.append('Not Normalized')
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [3]:
data= pd.read_csv('../data.csv')

In [4]:
##copying data
data1=data.copy()
### spliting data en X et Y
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']
### spliting the data to train and test
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)
## dicision tree (CART)
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)


In [5]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'Decision Tree', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [6]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")


----------
best parameters:  {'criterion': 'entropy', 'max_depth': 11}
----------
[[10743  4616]
 [  252 15155]]
----------
              precision    recall  f1-score   support

           0       0.98      0.70      0.82     15359
           1       0.77      0.98      0.86     15407

    accuracy                           0.84     30766
   macro avg       0.87      0.84      0.84     30766
weighted avg       0.87      0.84      0.84     30766

----------


In [7]:
grid_search_acc.cv_results_



{'mean_fit_time': array([0.30717945, 0.37300301, 0.42436683, 0.46276188, 0.49816763,
        0.50714326, 0.5445416 , 0.55401564, 0.6412847 , 0.62433064,
        0.64876604, 0.64377773, 0.68815851, 0.75747657, 0.86069787,
        0.37948406, 0.56349349, 0.69912994, 0.71857667, 0.70910048,
        0.73453474, 0.74500918, 0.8452394 , 0.90059114, 1.01877451,
        1.02974629, 1.03024495, 1.00481129, 0.90906847, 0.92159414]),
 'std_fit_time': array([0.00598335, 0.00299263, 0.01944792, 0.01296496, 0.00249445,
        0.00249314, 0.02393508, 0.00149727, 0.01795244, 0.02792418,
        0.00149679, 0.00847542, 0.0089767 , 0.08826566, 0.04787242,
        0.00149477, 0.04388285, 0.04687536, 0.03440809, 0.02493668,
        0.02642918, 0.0398941 , 0.04438162, 0.00698185, 0.05235958,
        0.00747991, 0.01196778, 0.00548673, 0.01246583, 0.01495624]),
 'mean_score_time': array([0.01695561, 0.01994801, 0.01795101, 0.01645529, 0.01745272,
        0.0169543 , 0.01994729, 0.02144265, 0.01944768, 0.01